In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
directory = "/media/sony/0CA83E99A83E80EE/Chris/Bocconi/Thesis/Data_copy/Data/"

In [3]:
df_main = pd.read_csv(directory + 'affiliates_dataV3.csv' )

In [4]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,aff_sales,aff_ex,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex,indus_im,skill_intens
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271792324.0,1.535549e+09,NaN
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71186832.0,1.461237e+09,NaN
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200605492.0,7.431215e+07,NaN
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30969497.0,1.302712e+06,NaN
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23017107.0,3.524300e+04,NaN


In [5]:
code_map = pd.read_csv(directory + 'country_code_map.csv')

In [6]:
country_code_map = code_map.set_index('country').code.to_dict()

In [7]:
country_code_map['Hong Kong, China'] = 'HKG'

In [8]:
add_map = {'Russia': 'RUS', 'Venezuela':'VEN','Hong Kong': 'HKG', 'Taiwan': 'TWN' }

In [9]:
country_code_map.update(add_map)

In [10]:
col_names = ['country',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]

In [11]:
df_concat = pd.DataFrame(columns = ['country','industry', 'aff_empl', 'host_code', 'year'])

for year in range(2009,2018):
    df_sales = pd.read_excel(directory + 'mofas-employment.xls', skiprows = 7, names=col_names, sheet_name = str(year))
    df_clean = df_sales.dropna(how = 'all', thresh =3).replace('(*)',0.0).replace('(D)', np.nan)
    df_long = pd.melt(df_clean,id_vars = 'country',var_name = 'industry',value_name ='aff_empl')
    df_long['host_code'] = df_long.country.map(country_code_map)
    df_long.dropna(subset= ['host_code'], inplace = True)
    df_long['year'] = int(year)
    df_concat = pd.concat([df_concat,df_long], sort = False)

In [12]:
empl_df = df_concat[['host_code','year','industry','aff_empl']]

In [13]:
empl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8064 entries, 0 to 1180
Data columns (total 4 columns):
host_code    8064 non-null object
year         8064 non-null object
industry     8064 non-null object
aff_empl     8064 non-null object
dtypes: object(4)
memory usage: 315.0+ KB


In [14]:
empl_df.head()

,host_code,year,industry,aff_empl
0,CAN,2009,1,1082.7
2,AUT,2009,1,41
3,BEL,2009,1,135.8
4,CZE,2009,1,72
5,DNK,2009,1,34.2


In [15]:
empl_df[empl_df.aff_empl =='M'].head()

,host_code,year,industry,aff_empl
704,MEX,2009,10,M
852,MEX,2009,12,M
1148,MEX,2009,16,M
704,MEX,2010,10,M
852,MEX,2010,12,M


In [16]:
value_map = {'A': 250, 'F': 750, 'G': 1750, 'H': 3750, 'I':7500, 'J':17500, 'K': 37500, 'L':75000,
            'M': 100000}

In [17]:
empl_df['aff_empl'] = empl_df['aff_empl'].replace(value_map)

/home/sony/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
empl_df[empl_df.aff_empl =='M']

/home/sony/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,host_code,year,industry,aff_empl


In [19]:
df_main = pd.merge(df_main, empl_df, how='left', on=['host_code','year', 'industry'])

In [20]:
df_concat = pd.DataFrame(columns = ['country','industry', 'aff_value_added', 'host_code', 'year'])

for year in range(2009,2018):
    df_sales = pd.read_excel(directory + 'mofas-value-added.xls', skiprows = 7, names=col_names, sheet_name = str(year))
    df_clean = df_sales.dropna(how = 'all', thresh =3).replace('(*)',0.0).replace('(D)', np.nan)
    df_long = pd.melt(df_clean,id_vars = 'country',var_name = 'industry',value_name ='aff_value_added')
    df_long['host_code'] = df_long.country.map(country_code_map)
    df_long.dropna(subset= ['host_code'], inplace = True)
    df_long['year'] = int(year)
    df_concat = pd.concat([df_concat,df_long], sort = False)
    

In [21]:
value_df = df_concat[['host_code','year','industry','aff_value_added']]

In [22]:
value_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8064 entries, 0 to 1180
Data columns (total 4 columns):
host_code          8064 non-null object
year               8064 non-null object
industry           8064 non-null object
aff_value_added    6869 non-null float64
dtypes: float64(1), object(3)
memory usage: 315.0+ KB


In [23]:
value_df.head()

,host_code,year,industry,aff_value_added
0,CAN,2009,1,113675.0
2,AUT,2009,1,5795.0
3,BEL,2009,1,23110.0
4,CZE,2009,1,5086.0
5,DNK,2009,1,8759.0


In [24]:
df_main = pd.merge(df_main, value_df, how='left', on=['host_code','year', 'industry'])

In [25]:
df_main.to_csv(directory + 'affiliates_dataV4.csv', index = False )

In [26]:
df_school = pd.read_csv(directory + 'Expected years of schooling (years).csv', encoding = "ISO-8859-1", skiprows =1,na_values ='..' )

In [27]:
df_school.drop(['HDI Rank (2017)','Unnamed: 57'], axis =1, inplace = True)

In [28]:
df_school.dropna(axis = 1,how ='all', inplace = True)

In [29]:
df_school.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017'],
      dtype='object')

In [30]:
df_school.dropna(thresh = 2, inplace = True)

In [31]:
df_school.head()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,2.6,2.9,3.2,3.6,3.9,4.2,4.6,4.9,5.2,...,8.9,9.2,9.5,9.8,10.0,10.2,10.4,10.4,10.4,10.4
1,Albania,11.6,11.8,10.7,10.2,10.1,10.2,10.3,10.6,10.7,...,12.2,12.3,13.1,13.8,14.7,14.8,14.8,14.8,14.8,14.8
2,Algeria,9.6,9.7,9.8,9.8,9.9,9.8,10.0,10.3,10.5,...,13.1,13.6,14.0,14.3,14.3,14.3,14.3,14.3,14.3,14.4
3,Andorra,10.8,10.8,10.8,10.8,10.8,10.8,10.8,10.8,10.8,...,11.7,11.7,11.7,11.7,13.5,13.5,13.5,13.5,13.5,13.5
4,Angola,3.8,3.6,3.5,3.7,3.8,3.9,4.0,4.1,4.2,...,8.5,9.0,8.6,9.5,9.9,10.3,10.7,11.0,11.4,11.8


In [32]:
school_melt = pd.melt(df_school, id_vars ='Country', var_name = 'year', value_name = 'yrs_school')

In [33]:
school_melt

,Country,year,yrs_school
0,Afghanistan,1990,2.6
1,Albania,1990,11.6
2,Algeria,1990,9.6
3,Andorra,1990,10.8
4,Angola,1990,3.8
5,Antigua and Barbuda,1990,NaN
6,Argentina,1990,13.2
7,Armenia,1990,10.7
8,Australia,1990,17.4
9,Austria,1990,14.0


In [34]:
UN_map = {'Andorra': 'AND', 'Bolivia (Plurinational State of)': 'BOL', 'Cabo Verde': 'CPV',
       'Congo (Democratic Republic of the)':'COD', 'Cuba':'CUB', 'Czechia': 'CZE',
       "Côte d'Ivoire":'CIV', 'Eswatini (Kingdom of)': 'SWZ', 'Guyana': 'GUY',
       'Hong Kong, China (SAR)': 'HKG', 'Iran (Islamic Republic of)': 'IRN', 'Kiribati':'KIR',
       "Korea (Democratic People's Rep. of)": 'PRK', 'Korea (Republic of)': 'KOR',
       "Lao People's Democratic Republic": 'LAO', 'Liechtenstein': 'LIE',
       'Micronesia (Federated States of)': 'FSM', 'Moldova (Republic of)': 'MDA',
       'Palestine, State of': 'PSE', 'San Marino': 'SMR', 'Sao Tome and Principe': 'STP',
       'South Sudan': 'SSD', 'Tanzania (United Republic of)':'TZA',
       'The former Yugoslav Republic of Macedonia':'MKD', 'Turkmenistan':'TKM',
       'Uzbekistan':'UZB', 'Venezuela (Bolivarian Republic of)':'VEN'}

In [35]:
country_code_map.update(UN_map)

In [36]:
school_melt['code'] = school_melt.Country.map(country_code_map)

In [37]:
school_melt[school_melt.code.isna()].Country.unique()

array([], dtype=object)

In [38]:
school_melt.year = school_melt.year.astype('int64')

In [39]:
home_school_df = school_melt[['code', 'year','yrs_school']]
home_school_df.columns = ['home_code', 'year','home_school']
home_school_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5404 entries, 0 to 5403
Data columns (total 3 columns):
home_code      5404 non-null object
year           5404 non-null int64
home_school    5142 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 126.7+ KB


In [40]:
home_school_df.head()

,home_code,year,home_school
0,AFG,1990,2.6
1,ALB,1990,11.6
2,DZA,1990,9.6
3,AND,1990,10.8
4,AGO,1990,3.8


In [41]:
host_school_df = school_melt[['code', 'year','yrs_school']]
host_school_df.columns = ['host_code', 'year','host_school']
host_school_df.head()

,host_code,year,host_school
0,AFG,1990,2.6
1,ALB,1990,11.6
2,DZA,1990,9.6
3,AND,1990,10.8
4,AGO,1990,3.8


In [42]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,aff_im,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex,indus_im,skill_intens,aff_empl,aff_value_added
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,271792324.0,1.535549e+09,NaN,NaN,NaN
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,71186832.0,1.461237e+09,NaN,NaN,NaN
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,200605492.0,7.431215e+07,NaN,NaN,NaN
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,30969497.0,1.302712e+06,NaN,NaN,NaN
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,23017107.0,3.524300e+04,NaN,NaN,NaN


In [43]:
df_main = pd.merge(df_main, home_school_df, how = 'left', on =['home_code','year'])

In [44]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,aff_parex,aff_parim,fdi_flow,fdi_position,indus_ex,indus_im,skill_intens,aff_empl,aff_value_added,home_school
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,271792324.0,1.535549e+09,NaN,NaN,NaN,15.5
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,71186832.0,1.461237e+09,NaN,NaN,NaN,15.5
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,200605492.0,7.431215e+07,NaN,NaN,NaN,15.5
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30969497.0,1.302712e+06,NaN,NaN,NaN,15.5
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,23017107.0,3.524300e+04,NaN,NaN,NaN,15.5


In [45]:
df_main = pd.merge(df_main, host_school_df, how = 'left', on =['host_code','year'])

In [46]:
df_main[df_main.host_school.notna()]

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,aff_parim,fdi_flow,fdi_position,indus_ex,indus_im,skill_intens,aff_empl,aff_value_added,home_school,host_school
380,USA,AFG,2000,1,10862.721256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8148732.0,780891.0,NaN,NaN,NaN,15.5,5.9
381,USA,AFG,2000,2,10862.721256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,501434.0,7986.0,NaN,NaN,NaN,15.5,5.9
382,USA,AFG,2000,3,10862.721256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,7647298.0,772905.0,NaN,NaN,NaN,15.5,5.9
383,USA,AFG,2000,4,10862.721256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5226127.0,237103.0,NaN,NaN,NaN,15.5,5.9
384,USA,AFG,2000,5,10862.721256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1369151.0,22063.0,NaN,NaN,NaN,15.5,5.9
385,USA,AFG,2000,6,10862.721256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,101237.0,NaN,NaN,NaN,NaN,15.5,5.9
386,USA,AFG,2000,7,10862.721256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5,5.9
387,USA,AFG,2000,8,10862.721256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5,5.9
388,USA,AFG,2000,9,10862.721256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,424102.0,NaN,NaN,NaN,NaN,15.5,5.9
389,USA,AFG,2000,10,10862.721256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,93857.0,NaN,NaN,NaN,NaN,15.5,5.9


In [47]:
home_school_df[home_school_df.home_code == 'USA']

,home_code,year,home_school
184,USA,1990,15.3
377,USA,1991,15.3
570,USA,1992,15.6
763,USA,1993,15.8
956,USA,1994,15.9
1149,USA,1995,15.9
1342,USA,1996,15.8
1535,USA,1997,15.8
1728,USA,1998,15.9
1921,USA,1999,15.8


In [48]:
df_edu_index = pd.read_csv(directory + 'Education Index.csv', encoding = "ISO-8859-1", skiprows =1, na_values ='..')

In [49]:
df_edu_index.head()

,HDI Rank (2017),Country,1990,Unnamed: 3,1991,Unnamed: 5,1992,Unnamed: 7,1993,Unnamed: 9,...,2013,Unnamed: 49,2014,Unnamed: 51,2015,Unnamed: 53,2016,Unnamed: 55,2017,Unnamed: 57
0,168,Afghanistan,0.122,NaN,0.133,NaN,0.145,NaN,0.156,NaN,...,0.398,NaN,0.406,NaN,0.410,NaN,0.410,NaN,0.415,NaN
1,68,Albania,0.584,NaN,0.589,NaN,0.558,NaN,0.543,NaN,...,0.732,NaN,0.733,NaN,0.733,NaN,0.742,NaN,0.745,NaN
2,85,Algeria,0.385,NaN,0.395,NaN,0.405,NaN,0.414,NaN,...,0.660,NaN,0.661,NaN,0.662,NaN,0.663,NaN,0.664,NaN
3,35,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.715,NaN,0.716,NaN,0.715,NaN,0.714,NaN,0.714,NaN
4,147,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.447,NaN,0.460,NaN,0.472,NaN,0.487,NaN,0.498,NaN


In [50]:
df_edu_index.dropna(axis =1, how = 'all', inplace = True)

In [51]:
df_edu_index.columns

Index(['HDI Rank (2017)', 'Country', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017'],
      dtype='object')

In [52]:
df_edu_index.drop(['HDI Rank (2017)'], axis =1, inplace = True)

In [53]:
df_edu_index.dropna(thresh = 2, inplace = True)

In [54]:
edu_index_melt = pd.melt(df_edu_index, id_vars ='Country', var_name ='year', value_name ='edu_index')

In [55]:
edu_index_melt['code'] = edu_index_melt.Country.map(country_code_map)

In [56]:
edu_index_melt[edu_index_melt.code.isna()]

,Country,year,edu_index,code


In [57]:
edu_index_melt.year = edu_index_melt.year.astype('int64')

In [58]:
home_edu_index = edu_index_melt[['code', 'year', 'edu_index']]
host_edu_index = edu_index_melt[['code', 'year', 'edu_index']]

In [59]:
home_edu_index.columns = ['home_code', 'year','home_edu_index']
host_edu_index.columns = ['host_code', 'year','host_edu_index']

In [60]:
home_edu_index.head()

,home_code,year,home_edu_index
0,AFG,1990,0.122
1,ALB,1990,0.584
2,DZA,1990,0.385
3,AND,1990,NaN
4,AGO,1990,NaN


In [61]:
host_edu_index.head()

,host_code,year,host_edu_index
0,AFG,1990,0.122
1,ALB,1990,0.584
2,DZA,1990,0.385
3,AND,1990,NaN
4,AGO,1990,NaN


In [62]:
df_main = pd.merge(df_main,home_edu_index, how = 'left',on=['home_code','year'] )

In [63]:
df_main = pd.merge(df_main, host_edu_index, how = 'left',on=['host_code','year'] )

In [64]:
df_main[df_main.home_edu_index.notna()].head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,fdi_position,indus_ex,indus_im,skill_intens,aff_empl,aff_value_added,home_school,host_school,home_edu_index,host_edu_index
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,271792324.0,1.535549e+09,NaN,NaN,NaN,15.5,NaN,0.854,NaN
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,71186832.0,1.461237e+09,NaN,NaN,NaN,15.5,NaN,0.854,NaN
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,200605492.0,7.431215e+07,NaN,NaN,NaN,15.5,NaN,0.854,NaN
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,30969497.0,1.302712e+06,NaN,NaN,NaN,15.5,NaN,0.854,NaN
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,23017107.0,3.524300e+04,NaN,NaN,NaN,15.5,NaN,0.854,NaN


In [65]:
df_main[df_main.host_edu_index.notna()].tail()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,fdi_position,indus_ex,indus_im,skill_intens,aff_empl,aff_value_added,home_school,host_school,home_edu_index,host_edu_index
77875,USA,ZWE,2017,16,12538.648513,80.0,53.33,38.73,58.83,53.33,...,NaN,632359.0,1308091.0,2.538755,NaN,NaN,16.5,10.3,0.903,0.558
77876,USA,ZWE,2017,17,12538.648513,80.0,53.33,38.73,58.83,53.33,...,NaN,NaN,NaN,NaN,NaN,NaN,16.5,10.3,0.903,0.558
77877,USA,ZWE,2017,18,12538.648513,80.0,53.33,38.73,58.83,53.33,...,NaN,NaN,NaN,NaN,NaN,NaN,16.5,10.3,0.903,0.558
77878,USA,ZWE,2017,19,12538.648513,80.0,53.33,38.73,58.83,53.33,...,NaN,NaN,NaN,NaN,NaN,NaN,16.5,10.3,0.903,0.558
77879,USA,ZWE,2017,20,12538.648513,80.0,53.33,38.73,58.83,53.33,...,NaN,NaN,NaN,NaN,NaN,NaN,16.5,10.3,0.903,0.558


In [66]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 60 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [67]:
df_main.to_csv(directory + 'affiliates_dataV5.csv', index = False )

In [68]:
df_main['dum_total'] = df_main.industry.apply(lambda x: int(1) if x == 1 else int(0))

In [69]:
df_main[df_main.dum_total ==1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3895 entries, 0 to 77880
Data columns (total 61 columns):
home_code            3895 non-null object
host_code            3895 non-null object
year                 3895 non-null object
industry             3895 non-null object
distance             3781 non-null float64
scoreproced          2099 non-null float64
minorinvest          2099 non-null float64
enfcontract          2386 non-null float64
paytax               2099 non-null float64
minorinvest_lag      1920 non-null float64
uncertainty          1610 non-null float64
home_gdp             3690 non-null float64
host_gdp             3386 non-null float64
couprisk             3060 non-null float64
couprisk_lag         3058 non-null float64
tariff_avg           2545 non-null float64
tariff_w_avg         2545 non-null float64
uncertainty_lag      1495 non-null float64
home_gdp_capita      3690 non-null float64
host_gdp_capita      3386 non-null float64
partner_directed     3690 non-null o

In [70]:
def by_indus_total(x):
    if x not in [1, 4,5,6,7,8,9,10]:
        return int(1)
    else:
        return int(0)

In [71]:
df_main['dum_indus_total'] = df_main.industry.apply(by_indus_total)

In [72]:
df_main[df_main.dum_indus_total ==1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46740 entries, 1 to 77899
Data columns (total 62 columns):
home_code            46740 non-null object
host_code            46740 non-null object
year                 46740 non-null object
industry             46740 non-null object
distance             45372 non-null float64
scoreproced          25188 non-null float64
minorinvest          25188 non-null float64
enfcontract          28632 non-null float64
paytax               25188 non-null float64
minorinvest_lag      23040 non-null float64
uncertainty          19320 non-null float64
home_gdp             44280 non-null float64
host_gdp             40632 non-null float64
couprisk             36720 non-null float64
couprisk_lag         36696 non-null float64
tariff_avg           30540 non-null float64
tariff_w_avg         30540 non-null float64
uncertainty_lag      17940 non-null float64
home_gdp_capita      44280 non-null float64
host_gdp_capita      40632 non-null float64
partner_directe

In [73]:
df_main[(df_main.dum_indus_total ==1) & (df_main.dum_total ==1)]

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,indus_im,skill_intens,aff_empl,aff_value_added,home_school,host_school,home_edu_index,host_edu_index,dum_total,dum_indus_total


In [78]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 62 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [74]:
def by_indus(x):
    if x not in [1, 3]:
        return int(1)
    else:
        return int(0)

In [75]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,indus_im,skill_intens,aff_empl,aff_value_added,home_school,host_school,home_edu_index,host_edu_index,dum_total,dum_indus_total
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,1.535549e+09,NaN,NaN,NaN,15.5,NaN,0.854,NaN,1,0
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,1.461237e+09,NaN,NaN,NaN,15.5,NaN,0.854,NaN,0,1
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,7.431215e+07,NaN,NaN,NaN,15.5,NaN,0.854,NaN,0,1
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,1.302712e+06,NaN,NaN,NaN,15.5,NaN,0.854,NaN,0,0
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,3.524300e+04,NaN,NaN,NaN,15.5,NaN,0.854,NaN,0,0


In [79]:
df_main['dum_indus'] = df_main.industry.apply(by_indus)

In [80]:
df_main.head()

,home_code,host_code,year,industry,distance,scoreproced,minorinvest,enfcontract,paytax,minorinvest_lag,...,skill_intens,aff_empl,aff_value_added,home_school,host_school,home_edu_index,host_edu_index,dum_total,dum_indus_total,dum_indus
0,USA,ABW,2000,1,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15.5,NaN,0.854,NaN,1,0,0
1,USA,ABW,2000,2,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15.5,NaN,0.854,NaN,0,1,1
2,USA,ABW,2000,3,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15.5,NaN,0.854,NaN,0,1,0
3,USA,ABW,2000,4,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15.5,NaN,0.854,NaN,0,0,1
4,USA,ABW,2000,5,3147.434733,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15.5,NaN,0.854,NaN,0,0,1


In [82]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77900 entries, 0 to 77899
Data columns (total 63 columns):
home_code            77900 non-null object
host_code            77900 non-null object
year                 77900 non-null object
industry             77900 non-null object
distance             75620 non-null float64
scoreproced          41980 non-null float64
minorinvest          41980 non-null float64
enfcontract          47720 non-null float64
paytax               41980 non-null float64
minorinvest_lag      38400 non-null float64
uncertainty          32200 non-null float64
home_gdp             73800 non-null float64
host_gdp             67720 non-null float64
couprisk             61200 non-null float64
couprisk_lag         61160 non-null float64
tariff_avg           50900 non-null float64
tariff_w_avg         50900 non-null float64
uncertainty_lag      29900 non-null float64
home_gdp_capita      73800 non-null float64
host_gdp_capita      67720 non-null float64
partner_directe

In [83]:
df_main.to_csv(directory + 'affiliates_dataV6.csv', index = False )